In [1]:
import sys
sys.path.append("..")

from datetime import datetime
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from lib.utils import get_samples_from_folder
from copy import deepcopy
import pandas as pd
from lib.features import WPTFeatureExtractor
from typing import Tuple
import pickle 

from models.metric_learning import (
    DataLoader,
    FeatureReducer,
    get_proba_per_sample,
    create_triplets_idx,
    TripletDataset,
    FeatureReducerScikit
)

from sklearn.pipeline import Pipeline

In [ ]:
plt.style.use("default")

In [ ]:
PARAMS = {
    "level": 6,
    "family": "db16",
    "d": 2,
}

In [ ]:
level = PARAMS["level"]
family = PARAMS["family"]
d = PARAMS["d"]

In [ ]:
PATH_TO_DATASET_1 = "/workspaces/project/raw/sequence_1/"
PATH_TO_DATASET_2 = "/workspaces/project/raw/sequence_2/"
PATH_MODELS = "../models/metric_learning/models_repo"

In [ ]:
fe = WPTFeatureExtractor(
    level = level,
    family = family,
    reduction = "energy",
    normalize = True
)

In [ ]:
for cable in range(1,5):
    print(cable)
    valid_date = datetime(2023, 1, 7, 10, 00)
    dataset_per_cable = get_samples_from_folder(PATH_TO_DATASET_2, valid_date)
    x = dataset_per_cable[f"c{cable}"][["y", "z"]].to_numpy()
    del dataset_per_cable
    x_wpt = fe.transform(x)
    fr = FeatureReducerScikit()
    fr.fit(x_wpt, d=d, num_epochs=50)

    output = {
        "feature_extraction": fe,
        "feature_reductor": fr,
        "params": PARAMS
    }

    filename = f"c{cable}_l{level}_f{family}_d{d}.pkl"

    with open(f"{PATH_MODELS}/{filename}", "wb") as h:
        pickle.dump(output, h, protocol=pickle.HIGHEST_PROTOCOL)